In [1]:
from loader.taps import TactileDataset

# train_dataset = TactileDataset('/home/hussain/me/projects/events_nn/data/morethan3500ev_lessthan_9deg/train', features='pol')
# val_dataset = TactileDataset('/home/hussain/me/projects/events_nn/data/morethan3500ev_lessthan_9deg/val', features='pol')

train_dataset = TactileDataset('/home/hussain/me/projects/tactile/data/extractions/morethan3500ev_lessthan_9deg/train', features='pol')
val_dataset = TactileDataset('/home/hussain/me/projects/tactile/data/extractions/morethan3500ev_lessthan_9deg/val', features='pol')


{'bag': '/media/kucarst3-dlws/drive1/tactile-data/data2', 'train_prop': 0.75, 'delta_t': 150000000.0, 'margin': -50000000.0, 'min_n_events': 3500, 'n_init_events': -1, 'downsample_factor': 2, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 20, 'possible_angles': [0.017453292519943295, 0.03490658503988659, 0.05235987755982989, 0.06981317007977318, 0.08726646259971647, 0.10471975511965978, 0.12217304763960307, 0.13962634015954636, 0.15707963267948966, 0.17453292519943295], 'theta': 'full', 'augmentations': [], 'filters': ['RemovePhiBiggerThan(max_phi=0.15707963267948966)', '<imports.EventArrayFilter.RemoveNEventsLessThan object at 0x7fb968c2a5c0>'], 'n': 2292, 'node_features': 'pol'}
{'bag': '/media/kucarst3-dlws/drive1/tactile-data/data2', 'train_prop': 0.75, 'delta_t': 150000000.0, 'margin': -50000000.0, 'min_n_events': 3500, 'n_init_events': -1, 'downsample_factor': 2, 'center': [180, 117], 'circle_rad': 85, 'case_span': 2660000000.0, 'N_examples': 20,

In [2]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=1)
val_loader = DataLoader(val_dataset, batch_size=1)

/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [3]:
import pytorch_lightning as pl
from models.eViT import eViT
from utils.lightning_modules import normality_trainer
from models.TactiGraph import TactiGraph

# model = eViT(170, 10)
model = TactiGraph()

In [4]:

trainer = pl.Trainer(gpus=1, max_epochs=10)
trainer.fit(normality_trainer(model, 1), train_loader, val_loader)

/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:466: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | TactiGraph | 45.6 K
----

Sanity Checking: 0it [00:00, ?it/s]

/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
